In [ ]:
!pip install compress_pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import math
import random
import multiprocessing
from datetime import datetime
import sys
import networkx as nx
from collections.abc import Iterable
import pickle
import matplotlib
import scipy as sp
import compress_pickle as pickle
from keras.layers import Dense, Add, Concatenate, Conv2D, Input, MaxPooling2D, Conv2DTranspose, Flatten
from keras import Model

In [ ]:
class MultiPente():
  def __init__(self, n_players=2, n_games=100, agent=None):
    self.boards = np.zeros(shape=(n_players, 19, 19, n_games))
    self.prev_boards = np.zeros(shape=(n_players, 19, 19, n_games))
    self.pairs = [dict(zip(list(range(0,n_players)), [0]*n_players)) for _ in range(n_games)]

    self.agent = agent

    self.n_players = n_players
    self.n_games = n_games
    self.completed = [False] * n_games
    self.graph = nx.DiGraph()
    self.history = {}
    self.outcomes = {}
    self.curr_player = 1

  def getLegalMoves(self):
    occupied = np.sum(self.boards, axis=0)
    mask = (occupied == 0)
    return mask

  def checkRow(self, seq):
    winning_seq = np.array([1, 1, 1, 1, 1])
    if 5 in np.convolve(winning_seq, seq, "valid"):
      return True
    else:
      return False

  def hash(self, a):
    out = pickle.dumps(a, compression="bz2")
    return out

  def getMoveIndices(self):

    if self.agent is None:

      values = np.random.uniform(0,1,size=(self.n_games, 19, 19))
      
      mask = self.getLegalMoves().reshape((self.n_games,19,19))
      self.mask = mask

      values = np.swapaxes(values, 0, -1).reshape((self.n_games,19,19))

      values = values * mask

      decision_indices = np.array([tuple(x) for x in zip(list(np.argmax(np.max(values, axis=2), axis=1)), np.argmax(np.max(values, axis=1), axis=1), list(range(self.n_games)))])

      return decision_indices
    
    else:
      # See http://tim.hibal.org/blog/alpha-zero-how-and-why-it-works/ for more information on UCB scores.

      values = self.agent.getValues(self.boards, self.pairs, self.last_player, self.graph)
      values = np.ones(shape=values.shape) / 2
      
      mask = self.getLegalMoves().reshape((self.n_games,19,19))
      self.mask = mask

      values_original = np.swapaxes(values, 0, -1).reshape((self.n_games,19,19))

      # Adding very small amount of noise to work around odd issue 
      # where agent tries to take same move over and over.

      values_noise = np.random.normal(0, .001, size=values_original.shape) + values_original
      values = values_noise * mask

      #TODO: Update "values" from agent guesses to UCB scores

      decision_indices = np.array([tuple(x) for x in zip(list(np.argmax(np.max(values, axis=2), axis=1)), np.argmax(np.max(values, axis=1), axis=1), list(range(self.n_games)))])

      return decision_indices

  def makeMoves(self, player_num):

    self.last_player = 0 if player_num == 1 else 1

    saved = None

    choices = self.getMoveIndices()

    zeros = np.zeros((choices.shape[0], choices.shape[1] + 1))
    zeros[:, 1:] = choices
    zeros[:, 0] = player_num

    choices = zeros.astype(np.int32)
    print("TOTAL MOVES MADE:", np.sum(self.boards))

    for game_num in range(self.n_games):
      if not self.completed[game_num]:
        seen_state=False
        self.last_board = self.boards[:,:,:,game_num].astype('i')
        self.boards[choices[game_num,:][0], choices[game_num,:][1], choices[game_num,:][2], choices[game_num,:][3]] = 1

        assert self.boards[choices[game_num,:][0], choices[game_num,:][1], choices[game_num,:][2], choices[game_num,:][3]] == 1
        
        self.curr_board = self.boards[:,:,:,game_num].astype('i')
        curr_choice = choices[game_num,:]

        if game_num in self.history.keys():
          self.history[game_num] += [curr_choice.tolist()[0:3]]
        else:
          self.history[game_num] = [curr_choice.tolist()[0:3]]

        x = curr_choice[1]
        y = curr_choice[2]

        padded_board = np.pad(self.curr_board, [(0,0), (4, 4), (4, 4)], mode='constant', constant_values=0)

        relevant = padded_board[player_num, x:x+9, y:y+9]

        diag = np.diagonal(relevant)
        alt_diag = np.fliplr(relevant).diagonal()

        horiz = relevant[4, :]
        vert = relevant[:, 4]

        last_node = self.hash((self.last_board.astype("int32"), self.pairs[game_num][self.last_player], self.pairs[game_num][player_num], self.last_player))

        if last_node not in self.graph:
          print(self.hash((self.last_board, self.pairs[game_num][self.last_player], self.pairs[game_num][player_num])) in self.graph)
          print(self.pairs[game_num][player_num], self.pairs[game_num][self.last_player])
          print(self.pairs)
          self.graph.add_node(last_node, visits=0, outcomes=0)

        for i, j in [[0,3], [3,0], [0,-3], [-3,0], [3,3], [-3,3], [3,-3], [-3,-3]]:
          if curr_choice[1] + i in range(0,19) and curr_choice[2] + j in range(0,19):
            if self.curr_board[curr_choice[0], curr_choice[1] + i, curr_choice[2] + j]:
              for player in range(0,self.n_players):
                if player != curr_choice[0]:
                  if self.curr_board[player, curr_choice[1] + i - int(i * 2/3), curr_choice[2] + j - int(j * 2/3)] and self.curr_board[player, curr_choice[1] + i - int(i * 1/3), curr_choice[2] + j - int(j * 1/3)]:
                    self.curr_board[player, curr_choice[1] + i - int(i * 2/3), curr_choice[2] + j - int(j * 2/3)] = 0
                    self.curr_board[player, curr_choice[1] + i - int(i * 1/3), curr_choice[2] + j - int(j * 1/3)] = 0

                    self.pairs[game_num][curr_choice[0]] += 1

        self.boards[:,:,:,game_num] = self.curr_board

        # Add last board state and current board state that stores num visits and sum of game outcomes to graph connected by edge 
        curr_node = self.hash((self.curr_board.astype("int32"), self.pairs[game_num][player_num], self.pairs[game_num][self.last_player], player_num))
        if curr_node not in self.graph:
          self.graph.add_node(curr_node, visits=0, outcomes=0)

        self.graph.add_edge(last_node, 
                            curr_node,
                            player=curr_choice[0],
                            move=curr_choice[1:3])       

        if self.checkRow(diag) or self.checkRow(alt_diag) or self.checkRow(horiz) or self.checkRow(vert):
          # Return 1 if game is completed on move
          self.completed[game_num] = True
          self.outcomes[game_num] = player_num

        if 5 <= max([value for key, value in self.pairs[game_num].items()]):
          # Return 1 if game is completed on move
          self.completed[game_num] = True
          self.outcomes[game_num] = player_num

        if self.completed[game_num] == True:
          self.graph.nodes[curr_node]["visits"] += 1
          self.graph.nodes[curr_node]["outcomes"] += player_num
          for node in nx.ancestors(self.graph, curr_node):

            self.graph.nodes[node]["visits"] += 1
            self.graph.nodes[node]["outcomes"] += player_num
          
    return self.completed  

  def getBoards(self):
    return self.boards

  def play(self):
    count = 0
    player = -1
    while not np.sum(self.completed) == self.n_games and count < 299:

      if player < self.n_players - 1:
        player+=1
      else:
        player=0

      fin_list = self.makeMoves(player)
      print("TAKING TURN", count)
      count += 1

    print("DONE PLAYING", self.n_games, "PENTE GAMES")
    self.reset()

  def reset(self):
    self.boards = np.zeros(shape=(self.n_players, 19, 19, self.n_games))
    self.pairs = [dict(zip(list(range(0, self.n_players)), [0]* self.n_players)) for i in range(self.n_games)]
    self.completed = [False for i in range(self.n_games)]
    self.curr_player = 1

In [ ]:
class Agent():
  def __init__(self):
    inp1 = Input(shape=(19,19,2))
    inp2 = Input(shape=(3))

    # Convolve
    conv1 = Conv2D(64, 5, activation="relu")(inp1)
    conv2 = Conv2D(64, 5, activation="relu")(conv1)
    conv3 = Conv2D(64, 3, activation="relu")(conv2)
    conv4 = Conv2D(64, 3, activation="relu")(conv3)
    conv5 = Conv2D(64, 3, activation="relu")(conv4)

    # Deconvolve
    deconv5 = Conv2DTranspose(64, 3, activation="relu")(conv5)
    deconv4 = Conv2DTranspose(64, 3, activation="relu")(deconv5)
    deconv3 = Conv2DTranspose(64, 3, activation="relu")(deconv4)
    deconv2 = Conv2DTranspose(64, 5, activation="relu")(deconv3)
    out1 = Conv2DTranspose(1, 5, activation="sigmoid")(deconv2)

    self.model = Model([inp1, inp2], [out1])
    self.model.compile(loss="mse", optimizer="adam")

  def getValues(self, boards, pairs, player_num, graph):

    board_states = np.swapaxes(boards, 0, -1)
    board_states = np.swapaxes(board_states, 1, -1)
    print(board_states.shape)
    board_states = list(board_states.astype("int32"))
    next_player = 0 if player_num == 1 else 1
    players_list = [player_num] * len(board_states)
    curr_pairs = [pair[next_player] for pair in pairs]
    last_pairs = [pair[player_num] for pair in pairs]
    inp2 = np.array(list(zip(curr_pairs, last_pairs, players_list)))
    node_list = [pickle.dumps(x, compression="bz2") for x in list(zip(board_states, last_pairs, curr_pairs, players_list))]

    # print(np.sum(board_states[0]))
    # print(board_states[0].shape)
    # print(last_pairs)
    # print(curr_pairs)
    # print(player_num)

    # for node in node_list:
    #   if node in graph:
    #     print("--------------------------")
    #     print("YYYYYYYYEEEEEEEETTTTTTTT")
    #     print("--------------------------")

    # print(list(map(graph.edges(data=True), node_list)))

    # UCB_score = 

    return self.model.predict([np.swapaxes(boards, 0, -1), inp2])



In [ ]:
agent = Agent()

num_batches = 100
# multi = MultiPente(n_players = 2, n_games = 5)
multi = MultiPente(n_players = 2, n_games = 10, agent=agent)

for batch in range(num_batches):
  print("BATCH", batch)

  multi.play()
  print(multi.boards.shape)

  with open(f"/content/drive/MyDrive/graph/graph.pkl", "wb") as f:
    pickle.dump(multi.graph, f)

Streaming output truncated to the last 5000 lines.
TAKING TURN 109
(10, 2, 19, 19)
TOTAL MOVES MADE: 983.0
TAKING TURN 110
(10, 2, 19, 19)
TOTAL MOVES MADE: 991.0
TAKING TURN 111
(10, 2, 19, 19)
TOTAL MOVES MADE: 998.0
TAKING TURN 112
(10, 2, 19, 19)
TOTAL MOVES MADE: 1007.0
TAKING TURN 113
(10, 2, 19, 19)
TOTAL MOVES MADE: 1016.0
TAKING TURN 114
(10, 2, 19, 19)
TOTAL MOVES MADE: 1023.0
TAKING TURN 115
(10, 2, 19, 19)
TOTAL MOVES MADE: 1029.0
TAKING TURN 116
(10, 2, 19, 19)
TOTAL MOVES MADE: 1037.0
TAKING TURN 117
(10, 2, 19, 19)
TOTAL MOVES MADE: 1045.0
TAKING TURN 118
(10, 2, 19, 19)
TOTAL MOVES MADE: 1053.0
TAKING TURN 119
(10, 2, 19, 19)
TOTAL MOVES MADE: 1058.0
TAKING TURN 120
(10, 2, 19, 19)
TOTAL MOVES MADE: 1065.0
TAKING TURN 121
(10, 2, 19, 19)
TOTAL MOVES MADE: 1070.0
TAKING TURN 122
(10, 2, 19, 19)
TOTAL MOVES MADE: 1077.0
TAKING TURN 123
(10, 2, 19, 19)
TOTAL MOVES MADE: 1082.0
TAKING TURN 124
(10, 2, 19, 19)
TOTAL MOVES MADE: 1088.0
TAKING TURN 125
(10, 2, 19, 19)
TOTAL MO

KeyboardInterrupt: ignored

In [ ]:
test_node = pickle.dumps((np.zeros((2,19,19), dtype="int32"), 0, 0, 1), compression="bz2")
test_node in multi.graph

True

In [ ]:
root = [n for n,d in multi.graph.in_degree() if d==0] 

In [ ]:
out = pickle.loads(root[0],compression="bz2")
print(out)
print(out[0].shape)

(array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0